In [1]:
!nvidia-smi
import sys
print(sys.prefix)
import os
HOME = os.getcwd()
print("HOME:", HOME)

gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

/work/van-speech-nlp/det2env
HOME: /work/van-speech-nlp/jindaznb/j-vis


In [2]:
# %cd detectron2
# !python -m pip uninstall .

In [3]:
# # install detectron2:
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
# # !pip install google-colab
# !python -m pip uninstall .
# # install dependencies: (use cu101 because colab has CUDA 10.1)
# !pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
# !pip install cython pyyaml==5.1
# !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

# !pip install torch
# !pwd
# !pip install --upgrade torch torchvision
# !pip3 install torch torchvision torchaudio

# !rm -rf detectron2
# !git clone https://github.com/facebookresearch/detectron2.git
# !python -m pip install -e detectron2

ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [4]:
# !pip install Pillow==9.5.0
# install dependencies: (use cu101 because colab has CUDA 10.1)
# !pip install cython pyyaml==5.1
# change pytorch version
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# import some common libraries
import numpy as np
import cv2
import random
# from google.colab.patches import cv2_imshow
# import some common detectron2 utilities

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog
from detectron2.utils.logger import setup_logger
setup_logger()

2.1.2+cu121 True
gcc (GCC) 4.8.5 20150623 (Red Hat 4.8.5-44)
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



<Logger detectron2 (DEBUG)>

In [5]:
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="a8fzziqukkQgsmGFnKgD")
# project = rf.workspace("forest-fire").project("forestfire2023")
# dataset = project.version(5).download("coco-mmdetection")
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="a8fzziqukkQgsmGFnKgD")
# project = rf.workspace("wildfire2024").project("forestfire2024")
# dataset = project.version(8).download("coco-mmdetection")

# from roboflow import Roboflow
# rf = Roboflow(api_key="a8fzziqukkQgsmGFnKgD")
# project = rf.workspace("wildfire2024").project("forestfire2024")
# dataset = project.version(2).download("coco-mmdetection")


In [6]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, f"{HOME}/Forestfire2024-1/train/_annotations.coco.json", f"{HOME}/Forestfire2024-1/train")
register_coco_instances("my_dataset_val", {}, f"{HOME}/Forestfire2024-1/valid/_annotations.coco.json", f"{HOME}/Forestfire2024-1/valid")
register_coco_instances("my_dataset_test", {}, f"{HOME}/Forestfire2024-1/test/_annotations.coco.json", f"{HOME}/Forestfire2024-1/test")

# register_coco_instances("my_dataset_train", {}, f"{HOME}/ForestFire2024-5/train/_annotations.coco.json", f"{HOME}/ForestFire2023-5/train")
# register_coco_instances("my_dataset_val", {}, f"{HOME}/ForestFire2023-5/valid/_annotations.coco.json", f"{HOME}/ForestFire2023-5/valid")
# register_coco_instances("my_dataset_test", {}, f"{HOME}/ForestFire2023-5/test/_annotations.coco.json", f"{HOME}/ForestFire2023-5/test")

# check register right version of dataset

In [7]:
#visualize training data
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

[02/09 01:31:57 d2.data.datasets.coco]: Loaded 471 images in COCO format from /work/van-speech-nlp/jindaznb/j-vis/Forestfire2024-1/train/_annotations.coco.json


In [8]:
#We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.
from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)